<font color= "green">Here the mother computer acts as a hub, collecting data from FED and then sending commands or events to the Arduino board.
Arduino receives those event markers from the mother computer and translates them into digital pulses for the TDT system</font>

## The data flow that we need to sync FEDs with TDT:
### FED3 to Computer: The FED3 devices send behavioral event data to the computer through their respective serial connections.
### Computer to Arduino: The computer processes this data and forwards relevant event information to the Arduino through a separate serial connection.
### Arduino to TDT System: The Arduino generates digital pulses or signals in response to the data it receives. These pulses are sent to the TDT system to mark the timing of behavioral events, enabling synchronization with neural recordings.

In [ ]:
#example of the code we can use to flash our arduino board

//int pulsePin = 10; // Use pin 10 for the LED or pulse output
//
//void setup() {
//  Serial.begin(115200);
//  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
//}
//
//void loop() {
//  if (Serial.available() > 0) {
//    String data = Serial.readStringUntil('\n');
//    
//    // Generate a pulse when data is received
//    digitalWrite(pulsePin, HIGH);  // Turn the LED on or send a pulse
//    delay(10);                     // 10 ms pulse duration
//    digitalWrite(pulsePin, LOW);   // Turn the LED off or end the pulse
//    
//    Serial.print("Received: ");
//    Serial.println(data);  // Print the received data to the Serial Monitor
//  }
//}



int pulsePin = 10; // Digital pin for output pulse to TDT

void setup() {
  Serial.begin(115200); // Initialize serial communication at 115200 baud rate
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output for the pulse
}

void loop() {
  if (Serial.available() > 0) { 
    // Check if data is available to read from the serial port
    char incomingByte = Serial.read(); // Read the incoming byte
    
    // Generate a pulse on pin 10 to send a signal to the TDT system
    digitalWrite(pulsePin, HIGH);  // Send a HIGH signal (pulse) to TDT
    delay(10);                     // 10 ms pulse duration (adjust if needed)
    digitalWrite(pulsePin, LOW);   // End the pulse (set pin 10 to LOW)

    // (Optional) Print the received byte for debugging
    Serial.print("Received: ");
    Serial.println(incomingByte);
  }
}


In [1]:
# this code should send data to Google spreadsheet and at the same time to Arduino board

import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(data, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    arduino_ser.write(data.encode('utf-8'))
    print(f"Sent to Arduino: {data}")

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        # to sync all the FED units with each other, I tend not to rely on FED clock,
        # Ignore the first field (timestamp from FED device) and use the computer's timestamp
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        # Assuming the data matches the order of the remaining column_headers
        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Append the row to Google Sheet
            sheet.append_row([timestamp] + data_list)
            
            # Send relevant data to Arduino
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM12"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


Worksheet 'Port_COM12' not found. Creating a new one.


# I noticed a delay between FEd logging the data and LED blinking on arduino, here I am trying to get around it by sending the pulse to Arduino first and then to spreadsheet

In [1]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(data, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    arduino_ser.write(data.encode('utf-8'))
    print(f"Sent to Arduino: {data}")

def log_to_google_sheet(sheet, timestamp, data_list):
    """
    Append the data to Google Sheets after sending it to Arduino.
    """
    sheet.append_row([timestamp] + data_list)

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino first
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
            
            # Then log the data to Google Sheets
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM12","COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


Worksheet 'Port_COM16' not found. Creating a new one.
Worksheet 'Port_COM12' not found. Creating a new one.
Data from COM12: 8/22/2024 10:20:37,22.85,52.42,1.16.3,FR1,8,4.19,NaN,1,Right,Left,4,65,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:05,22.14,54.37,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.01
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:07,22.16,54.39,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,2,0,0,NaN,NaN,0.03
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:57,22.33,53.67,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,3,0,0,NaN,NaN,0.26
Sent to Arduino: Left
Data from COM12: 8/22/2024 10:21:30,22.85,52.37,1.16.3,FR1,8,4.19,NaN,1,Right,Left,4,66,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:58,22.34,53.67,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,4,0,0,NaN,NaN,0.06
Sent to Arduino: Left
Data from COM12: 8/22/2024 10:21:33,22.87,52.37,1.16.3,FR1,8,4.20,NaN,1,Right,Left,4,67,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Dat

# to make separate event signals to the TDT we need to adjust the python script that gets FED readout and also how Arduino signals TDT

In [ ]:

# I wil replace this snipet with the orginal send_to_arduino function
def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")


#then I will flash the arduino code with this script 
int pulsePin = 10; // Pin for output pulse

void setup() {
  Serial.begin(115200);
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
}

void loop() {
  if (Serial.available() > 0) {
    char eventType = Serial.read(); // Read the event type identifier

    switch(eventType) {
      case 'R': // Right poke event
        generatePulse(1); // Single pulse
        break;
      case 'L': // Left poke event
        generatePulse(2); // Double pulse
        break;
      case 'P': // Pellet intake event
        generatePulse(3); // Triple pulse
        break;
      case 'W': // Left with pellet event
        generatePulse(4); // Quadruple pulse
        break;
      case 'Q': // Right with pellet event
        generatePulse(5); // Quintuple pulse
        break;
    }
  }
}

void generatePulse(int pulseCount) {
  for (int i = 0; i < pulseCount; i++) {
    digitalWrite(pulsePin, HIGH);  // Send a pulse
    delay(10);                     // 10 ms pulse duration
    digitalWrite(pulsePin, LOW);   // End the pulse
    delay(100);                    // Wait 100 ms between pulses
  }
}



# code below testing distinct TTL signals
# code below is tested and sends TTL signals according to the behavioural event
# This code requires internet connection since it needs to log the data on google spreadsheet as well

In [2]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")

def log_to_google_sheet(sheet, timestamp, data_list):
    """
    Append the data to Google Sheets after sending it to Arduino.
    """
    sheet.append_row([timestamp] + data_list)

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino first
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
            
            # Then log the data to Google Sheets
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


SerialException: could not open port 'COM38': PermissionError(13, 'Access is denied.', None, 5)

# code below is the same code as above, but does not send data to the spreadsheet so should not need internet connection

In [1]:
import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino
            event_type = data_list[9]  # "Event" field contains event type
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


Data from COM16: 8/22/2024 17:33:05,24.56,51.34,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,44,9,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:11,24.56,51.33,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,45,9,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:13,24.56,51.37,1.16.3,FR1,20,4.19,NaN,1,Right,Left,9,46,9,0,NaN,NaN,0.06
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:14,24.57,51.35,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,47,9,0,NaN,NaN,0.05
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:16,24.56,51.34,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,48,9,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:17,24.58,51.34,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,49,9,0,NaN,NaN,0.15
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:20,24.58,51.36,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,50,9,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 17:33:27,24.59,51.33,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,51,9,0,NaN,NaN,0.0

# to connect multiple FEDs with their own TTL signals, let's try the code below

In [1]:
def send_to_arduino(fed_id, event_type, arduino_ser):
    """
    Send the event data and FED identifier to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    command = f"{fed_id}_{event_type}"  # Create a unique command for each FED and event
    arduino_ser.write(command.encode('utf-8'))
    print(f"Sent to Arduino: {command}")

def read_from_port(ser, worksheet_name, arduino_ser, fed_id):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            event_type = data_list[9]  # "Event" field contains event type
            send_to_arduino(fed_id, event_type, arduino_ser)  # Send FED ID and event type to Arduino
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")


# and then I will need to flash the arduino with this script
int pulsePin = 10; // Pin for output pulse

void setup() {
  Serial.begin(115200);
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
}

void loop() {
  if (Serial.available() > 0) {
    String command = Serial.readStringUntil('\n'); // Read the full command
    
    if (command == "FED1_Left") {
        generatePulse(1);  // Example: Single pulse for FED1 Left
    } else if (command == "FED1_Right") {
        generatePulse(2);  // Example: Double pulse for FED1 Right
    } else if (command == "FED2_Left") {
        generatePulse(3);  // Example: Triple pulse for FED2 Left
    } else if (command == "FED2_Right") {
        generatePulse(4);  // Example: Quadruple pulse for FED2 Right
    } // Continue with other cases
  }
}

void generatePulse(int pulseCount) {
  for (int i = 0; i < pulseCount; i++) {
    digitalWrite(pulsePin, HIGH);  // Send a pulse
    delay(10);                     // 10 ms pulse duration
    digitalWrite(pulsePin, LOW);   // End the pulse
    delay(100);                    // Wait 100 ms between pulses
  }
}


SyntaxError: invalid syntax (1957769390.py, line 31)

# to test the delay between sensor and serial connection I will try the code below

In [1]:
import serial
import threading
import datetime

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    send_time = datetime.datetime.now()  # Record time before sending
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type} at {send_time}")

    # Wait for Arduino response
    try:
        arduino_response = arduino_ser.readline().decode('utf-8', errors='replace').strip()
        receive_time = datetime.datetime.now()  # Record time when response is received
        print(f"Arduino responded at {receive_time}: {arduino_response}")

        # Calculate the delay
        delay = receive_time - send_time
        print(f"Delay between sending and receiving: {delay.total_seconds()} seconds")
    except UnicodeDecodeError as e:
        print(f"Error decoding Arduino response: {e}")


def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")
        data_list = data_list[1:]  # Skip the FED device timestamp

        print(f"Data from {ser.port}: {data}")

        if len(data_list) == 17:  # Adjusted to the correct length
            event_type = data_list[9]  # Assuming "Event" is at index 9
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match expected length.")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


# and flash the Arduino board with this code
# int ledPin = 10; // Pin for output pulse

# void setup() {
#   Serial.begin(115200);
#   pinMode(ledPin, OUTPUT); // Set pin 10 as an output
# }

# void loop() {
#   if (Serial.available() > 0) {
#     char eventType = Serial.read(); // Read the event type identifier

#     // Blink the LED
#     digitalWrite(ledPin, HIGH);  
#     delay(10);  // 10 ms pulse duration
#     digitalWrite(ledPin, LOW);   

#     // Send a response back to Python
#     Serial.print("Blink done for event: ");
#     Serial.println(eventType);
#   }
# }


Data from COM16: 8/22/2024 17:56:05,24.75,51.44,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,100,9,0,NaN,NaN,0.00
Sent to Arduino: Left at 2024-08-22 17:17:48.988360
Arduino responded at 2024-08-22 17:17:48.991372: Ev�Arduino ready
Delay between sending and receiving: 0.003012 seconds
Data from COM16: 8/22/2024 17:56:09,24.75,51.51,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,101,9,0,NaN,NaN,0.00
Sent to Arduino: Left at 2024-08-22 17:17:53.098080
Arduino responded at 2024-08-22 17:17:53.100037: Event received:
Delay between sending and receiving: 0.001957 seconds
Data from COM16: 8/22/2024 17:56:11,24.75,51.48,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,102,9,0,NaN,NaN,0.00
Sent to Arduino: Left at 2024-08-22 17:17:54.572925
Arduino responded at 2024-08-22 17:17:54.572925: L
Delay between sending and receiving: 0.0 seconds
Data from COM16: 8/22/2024 17:56:14,24.74,51.30,1.16.3,FR1,20,4.20,NaN,1,Right,Left,9,103,9,0,NaN,NaN,0.00
Sent to Arduino: Left at 2024-08-22 17:17:58.106508
Arduino responded at 2024